In [2]:
import sys
import os
import pandas as pd
from pathlib import Path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.etl.infrastructure.db_connector import DBConnector # noqa
from src.etl.domain.entities import Customer # noqa
from src.etl.infrastructure.real_customer_repository import RealCustomerRepository # noqa

"""
Lire le fichier clients.csv et regrouper les clients par ville.
"""
def customers_csv_to_df():
    filepath = Path('../../../data_src/clients.csv')
    df = pd.read_csv(filepath)
    group_by_city = df.groupby('ville').size().reset_index(name='counts')
    return group_by_city
display(customers_csv_to_df().head(5))


,ville,counts
0,Bordeaux,10
1,Lyon,10
2,Marseille,10
3,Paris,10
4,Toulouse,10


In [3]:
# Récupérer le prix moyen par catégorie de produit
def get_average_price_by_category():
    filepath = Path('../../../data_src/produits.json')
    df = pd.read_json(filepath)
    avg_price_by_category = df.groupby('categorie')['prix'].mean().reset_index(name='average_price')
    avg_price_by_category = avg_price_by_category.sort_values(by='average_price', ascending=False)
    return avg_price_by_category

display(get_average_price_by_category().head(2))

,categorie,average_price
1,Catégorie B,35.0
0,Catégorie A,32.5


In [23]:
from pathlib import Path
import os
import sys
import pandas as pd
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

def get_db_sales():
    from src.etl.infrastructure.db_connector import DBConnector
    conn = DBConnector()

    result = conn.execute_query("SELECT * FROM ventes")
    if result is None or len(result) == 0:
        with open(Path('../../../data_src/ventes.sql')) as f:
            sql_commands = f.read()
            conn.execute_query(sql_commands)
        result = conn.execute_query("SELECT * FROM ventes")
    
    if result is not None:
        result = [row[:5] for row in result]
        columns = ['id_client', 'id_produit', 'quantite', 'date_vente', 'montant']
        sales_df = pd.DataFrame(result, columns=columns, index=None)
        return sales_df

    else:
        print("No data found in 'ventes' table.")

sales_df = get_db_sales()
display(sales_df.head(5))

,id_client,id_produit,quantite,date_vente,montant
0,1,5,5,3,2024-07-15
1,2,40,6,1,2024-05-27
2,3,4,19,4,2024-07-11
3,4,20,15,2,2024-02-16
4,5,22,10,2,2024-02-19


In [22]:
from pathlib import Path
import pandas as pd

def aggregate_data():
    c_filepath = Path('../../../data_src/clients.csv')
    c_df = pd.read_csv(c_filepath)

    p_filepath = Path('../../../data_src/produits.json')
    p_df = pd.read_json(p_filepath)
    sales_df = get_db_sales()

    aggr_df = pd.merge(sales_df, c_df, on='id_client', how='left')
    aggr_df = pd.merge(aggr_df, p_df, on='id_produit', how='left')
    return aggr_df



aggr_df = aggregate_data()
print("Aperçu des données agrégées avant nettoyage:")
display(aggr_df.head(5))

def clean_aggregated_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.fillna(value={col: '' for col in df.select_dtypes(include='object').columns})
    df = df.dropna()

    return df

clean_aggr = clean_aggregated_data(aggr_df)
print("Aperçu des données agrégées après nettoyage:")
display(clean_aggr.head(5))

Aperçu des données agrégées avant nettoyage:


,id_client,id_produit,quantite,date_vente,montant,nom,prenom,email,ville,pays,libelle,categorie,prix
0,1,5,5,3,2024-07-15,Martin,Alice,alice.martin@email.com,Paris,France,Produit 5,Catégorie A,20.0
1,2,40,6,1,2024-05-27,Dupont,Julien,julien.dupont@email.com,Lyon,France,NaN,NaN,NaN
2,3,4,19,4,2024-07-11,Bernard,Claire,claire.bernard@email.com,Marseille,France,Produit 4,Catégorie B,17.5
3,4,20,15,2,2024-02-16,Petit,Thomas,thomas.petit@email.com,Toulouse,France,Produit 20,Catégorie B,57.5
4,5,22,10,2,2024-02-19,Robert,Sophie,sophie.robert@email.com,Bordeaux,France,NaN,NaN,NaN


Aperçu des données agrégées après nettoyage:


,id_client,id_produit,quantite,date_vente,montant,nom,prenom,email,ville,pays,libelle,categorie,prix
0,1,5,5,3,2024-07-15,Martin,Alice,alice.martin@email.com,Paris,France,Produit 5,Catégorie A,20.0
2,3,4,19,4,2024-07-11,Bernard,Claire,claire.bernard@email.com,Marseille,France,Produit 4,Catégorie B,17.5
3,4,20,15,2,2024-02-16,Petit,Thomas,thomas.petit@email.com,Toulouse,France,Produit 20,Catégorie B,57.5
5,6,17,1,4,2024-04-10,Martin,Alice,alice.martin@email.com,Paris,France,Produit 17,Catégorie A,50.0
6,7,10,17,2,2024-06-08,Dupont,Julien,julien.dupont@email.com,Lyon,France,Produit 10,Catégorie B,32.5
